## Input Analysis

In [12]:
import numpy as np
import scipy.stats as st
import pandas as pd
import datetime as dt


data = pd.read_csv('TonerItDown.csv')
data = data[data['Time of day']<24]
n = len(data)

#Create rate table for Request generation
emp_rate = data.groupby(data['Time of day'].apply(np.floor)).size() / 60
emp_rate = emp_rate.values
coeffs = np.polyfit(np.arange(3,22),emp_rate[3:22],deg = 2)
fitted_rate = np.zeros(24)
fitted_rate[3:22] = coeffs[0]*np.arange(3,22) ** 2 + coeffs[1]*np.arange(3,22) + coeffs[2]
fitted_rate[[0,1,2,22,23]] = np.mean(emp_rate[[0,1,2,22,23]])
rates = pd.DataFrame(data = fitted_rate,columns = ['fitted rate'])

#Paramaters for Initial Diagnosis Time fitted to the normal distribution 
param_diagnose_a = st.norm.fit(data[data['Request location'].isin
                         (['BC_1','BC_10','BC_4','BC_5','BC_6','BC_7','BC_8'])]['Initial diagnose time'])
param_diagnose_b = st.norm.fit(data[data['Request location'].isin(['BC_2','BC_3','BC_9'])]
                               ['Initial diagnose time'])

# Paramaters for the beta distribution fit for Onsite Repair Time
data_on_site = data['On-site repair time'][data['Needs Replacement?'] != 'yes']
n_on_site = len(data_on_site)
params_repair_beta = st.beta.fit(data_on_site)

#Empirical Probabilities of a call orginating at a BC
p_hat = data.groupby(by = 'Request location')['Initial diagnose time'].size() / n
std = np.sqrt(p_hat * (1-p_hat))
probabilities = p_hat.values.tolist()

# Create Python objects from data given in problem specification
Distances = {
            "BC1":{"BC2": 20,"BC3": 30,"BC4": 45,"BC5": 50,"BC6": 50,"BC7": 60,"BC8": 55,"BC9": 60,"BC10": 70,"Dispatch":45},
            "BC2":{"BC1": 20,"BC3": 10,"BC4": 25,"BC5": 30,"BC6": 50,"BC7": 60,"BC8": 55,"BC9": 60,"BC10": 70,"Dispatch": 45},
            "BC3":{"BC1": 30,"BC2": 10,"BC4": 15,"BC5": 20,"BC6": 40,"BC7": 50,"BC8": 45,"BC9": 50,"BC10": 60,"Dispatch": 35},
            "BC4":{"BC1": 45,"BC2": 25,"BC3": 15,"BC5": 5 ,"BC6": 55,"BC7": 65,"BC8": 60,"BC9": 65,"BC10": 75,"Dispatch": 50},
            "BC5":{"BC1": 50,"BC2": 30,"BC3": 20,"BC4": 5 ,"BC6": 60,"BC7": 70,"BC8": 65,"BC9": 70,"BC10": 80,"Dispatch": 55},
            "BC6":{"BC1": 50,"BC2": 50,"BC3": 40,"BC4": 55,"BC5": 60,"BC7": 10,"BC8": 5 ,"BC9": 10,"BC10": 20,"Dispatch": 25},
            "BC7":{"BC1": 60,"BC2": 60,"BC3": 50,"BC4": 65,"BC5": 70,"BC6": 10,"BC8": 15,"BC9": 20,"BC10": 10,"Dispatch": 35},
            "BC8":{"BC1": 55,"BC2": 55,"BC3": 45,"BC4": 60,"BC5": 65,"BC6": 5 ,"BC7": 15,"BC9": 5 ,"BC10": 15,"Dispatch": 30},
            "BC9":{"BC1": 60,"BC2": 60,"BC3": 50,"BC4": 65,"BC5": 70,"BC6": 10,"BC7": 20,"BC8": 5 ,"BC10": 10,"Dispatch": 35},
            "BC10":{"BC1":70,"BC2": 70,"BC3": 60,"BC4": 75,"BC5": 80,"BC6": 20,"BC7": 10,"BC8": 15,"BC9" : 10,"Dispatch": 45}}

BusinessCenters = ["BC1", "BC2", "BC3", "BC4", "BC5", "BC6", "BC7", "BC8", "BC9", "BC10"]

BC_probabilities = (("BC1", 0.039), ("BC2", 0.082), ("BC3", 0.108), ("BC4", 0.135),
                    ("BC5", 0.118), ("BC6", 0.055), ("BC7", 0.124), ("BC8", 0.058),
                    ("BC9", 0.137), ("BC10", 0.142))

In [17]:
class Request:
    def __init__(self, initialized_time):
        self.location = np.random.choice(BusinessCenters,1,p=probabilities)[0]
        self.status = "Waiting"
        self.initialized_time = None
        self.times = []
        self.assigned_mechanic = None
        self.assigned_van = None
        self.first_req(initialized_time)
    
    #Creates random interarrival time for the request to be created, appends time to times
    def first_req(self,initialized_time):
        tmp = initialized_time + dt.timedelta(minutes=
                                (60.0 / float(np.random.poisson(rates[rates.index == initialized_time.hours()],1)[0])))
        self.initialized_time = tmp
        self.times.append(tmp)
        
    #Check if this repair can be done on-site and update the status accordingly
    def update_waiting_status(self):
        if self.status == "Waiting":
            self.status = np.random.choice(["Onsite","Replace"],1,p=[.4,.6])[0] # fix these probs
    
    # Mechanic is found in sim_main. Assigns the found mechanic to the request
    def assign_mechanic (self, mechanicid) :
        self.assigned_mechanic = mechanicid     
       
    #function that calculates the time required to travel between the customers location and [loc]
    def add_travel_time(self, now, loc) :
        self.times.append(now + dt.timedelta(minutes=Distances[self.location][loc]/60))
        
    def gen_diagnose_time(self,now):
        if self.location in ['BC_2','BC_3','BC_9']:
            self.times.append(now + dt.timedelta(minutes=random.normal(args = param_diagnose_b)))
        else: self.times.append(now + dt.timedelta(minutes=random.normal(args = param_diagnose_a)))
   
    def gen_onsite_repair_time(self,now):
        self.times.append(now + dt.timedelta(minutes = random.beta(args = params_repair_beta)))

    #Random time for a van to replace the copier
    def gen_van_replace_time(now) :
        self.times.append(now + dt.timedelta(minutes= st.random.triangular(20,30,60) ))
    
    #Van if found in sim_main. Assigns the found vanid to the request
    def assign_van (self,vanid) :
        self.assigned_van = vanid

        
class Van:
    def __init__(self,id):
        self.loc = 'Dispatch'
        self.busy = False
        self.id = id
        
        
class Mechanic:
    def __init__(self,id):
        self.loc = 'Dispatch'
        self.busy = False
        self.id = id

[datetime.datetime(2019, 11, 21, 16, 55, 56, 32824)]

In [ ]:
class Simulation:
    def __init__(self, n_mechanics, n_vans, n_runs):
        self.n_mechanics = n_mechanics
        self.n_vans = n_vans
        self.n_runs = n_runs
        self.mechanics = []
        self.vans = []
        self.requests = []

    # Find the nearest free worker (of given type) of out of list and set to busy
    def find_nearest(self, worker_type):
        request_loc = self.requests[0].location      #location of the request
        if worker_type == mechanics:
            loop_list = self.mechanics
        else:
            loop_list = self.vans
       
        closest_distance = 100
        assigned_worker = None
       
        for worker in loop_list:
            if worker.busy == False:
                current_worker = worker
                current_location = worker.loc
                current_distance = Disances[request_loc][current_location]
            if current_distance < closest_distance:
                assigned_worker = worker
                closest_distance = current_distance
        return(assigned_worker)
    
    #Free set the busy boolean for the worker assigned to the first request to false
    def free_worker(self, worker_type):
        if(worker_type == "Mechanic"):
            for mechanic in self.mechanics:
                if(mechanic.id == self.requests[0].assigned_mechanics):
                    mechanic.busy = False
        elif(worker_type == "Van"):
            for van in self.vans:
                if(van.id == self.requests[0].assigned_van):
                    van.busy = False
    
    # Moves the request at the first index, to the index of the first element with the last element of times
    # greater than that of the last element of the current first requests time list
    def move_first(self):
        tmp_req = self.requests.pop(0)
        index = 0
        while(self.requests[index].times[-1] < tmp_req.times[-1]):
            next_req += 1
        self.requests.insert(tmp_req, index)
    
    def main(self):
            for m in range(1,self.n_mechanics):
                self.mechanics.append(Mechanic(m))

            for v in range(1,self.n_vans)
                self.vans.append(Van(v))

            current_time =  #Beginning Date Time
            end_time = # time to end simulation
            first_request = Request(current_time, x) 
            self.requests.append(first_request)
            while(current_time < end_time): #Extend buisness hours or no
                update_num = len((self.requests[0]).times)

                if(update_num == 1):                                      # We need to move a mechanic
                    mech = find_nearest(mechanics, self.requests[0].loc)  # Find nearest free mechanic
                    self.requests[0].assign_mechanic(mech.id)             # Give the request an assigned mechanic
                    self.requests[0].add_travel_time(mech.loc)            # Add the mechanics travel time
                    self.requests.prepend(Request(current_time))          # Create a new next request
                    self.requests.move_first()
                    self.requests.move_first()

                elif(update_num == 2):                                    # We need to diagnose the problem
                    self.requests[0].gen_diagnose_time()                  # Randomly generate diagnosis time
                    self.requests[0].update_waiting_status()              # Randomly generate repair / replace
                    self.requests.move_first()

                elif(update_num == 3):                                    # Need to either repair or replace 
                    if(self.requests[0].status == "Onsite"):              # It's a repair
                        self.requests[0].gen_onsite_repair_time()         # Generate a repair time
                    else:                                                 # Need to request a van
                        van = find_nearest(vans, self.requests[0].loc)    # Find a van
                        current_request.assign_van(van.id)                # Assign the van to the request
                        self.requests[0].add_travel_time(van.loc)         # Add travel time to site
                        self.free_worker("Mechanic")
                    self.requests.move_first()

                # Do we free the worker now in case of van?
                elif(update_num == 4):                                   # Finish off the van replacement of  
                    if(self.requests[0].status == "Onsite"):
                        self.free_worker("Mechanic")                     # Worker finished repair, now free
                        tmp_request = self.requests.pop(0)               # Remove the current request
                        self.requests.append(tmp_request)                # Move to the end of the list, done with it
                    else:
                        self.requests[0].gen_van_replace_time()
                        self.requests.move_first()

                elif(update_num == 5): # Wait for van to finish 
                    self.requests[0].add_travel_time('Dispatch')
                    self.requests.move_first()
                    # end van task

                elif(update_num == 6):
                    self.requests[0].add_van_swap_time()
                    self.requests.move_first()

                elif(update_num == 7):
                    self.free_worker(self.requests[0].assigned_van, vans)
                    #move worker to the end

                current_time = self.requests[0].times[len(self.requests[0].times) - 1]     

In [3]:
n_runs = 1
max_mechanics = 10
max_vans = 10 

# Run simulation n times, storing the results in a df
means = []
variances = []
for m in range(1,max_mechanics):
    for v in range(1, max_vans):
        for x in range(1, n_runs):
            sim = Simulation(m + 1, v + 1)
            df = pd.DataFrame([y.as_dict() for y in Simulation.requests])
            df['Run'] = x
            df['n_Mechanics'] = m + 1
            df['n_Vans'] = v +1
            if(x==0):
                results = df
            else:
                results.append(df)
        response = []
        replace = []
        for index, row in df.iterrows():
            response.append(row['times'][1] - row['times'][0])
        mean = []
        var = []
        for c in results.columns:
            mean.append(results[c].mean())
            var.append(results[c].var())
        means.append(mean)
        variances.append(var)


NameError: name 'Simulation' is not defined